In [7]:
import os

from tqdm import tqdm

import pandas as pd
import numpy as np

from run_kmeans import RunKmeans

from os.path import join, splitext
from numpy import min, mean, sum

In [9]:
pwd = os.getcwd()
datadir = join(pwd, 'data/')
outdir = join(pwd, 'out/')
files = os.listdir(datadir)
filename = 'boston.csv'
file = join(datadir, filename)
num_clusters = [
    5,
]
inits = [
    'random',
    'k-means++',
    'coc',
    'orss',
]

In [3]:
dfs = []
for file in tqdm(files[:]):
    filepath = join(datadir, file)
    comparator = RunKmeans(
        n_clusters=5,
        num_exp=20,
        file=filepath,
    )
    inertias, times, iters = comparator.run_kmeans()
#     print(file)
    rows = []
    for k in inertias:
        for inertia in inertias[k]:
            if type(inertia) == type(np.float64(1.0)):
                continue
            elif type(inertia) == type(np.int64(1.0)):
                continue
            else:
                raise TypeError(inertia, 'not float or int')
        row = []
        row.append(k)
#         print(k, mean(inertias[k]), mean(times[k]))
        row.append(round(mean(inertias[k]), 2))
        row.append(round(min(inertias[k]), 2))
        row.append(round(mean(times[k]), 2))
        row.append(round(min(times[k]), 2))
        row.append(round(mean(iters[k]), 2))
        row.append(round(min(iters[k]), 2))
        rows.append(row)
        columns=[
            'Initialization', 
            'Average Inertia', 
            'Minimum Inertia',
            'Average Time',
            'Minimum Time',
            'Average Iteration',
            'Minimum Iteration',
        ]
    df = pd.DataFrame(rows, columns=columns)
    dfs.append(df)

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [02:45<00:00, 18.36s/it]


In [4]:
for i, df in enumerate(dfs):
    print(files[i])
    print(df.head())

airlines.csv
  Initialization  Average Inertia  Minimum Inertia  Average Time  \
0         random     5.788610e+12     5.788610e+12          1.68   
1      k-means++     5.895772e+12     5.724391e+12          1.34   
2           orss     5.833118e+12     5.724391e+12          1.29   
3            coc     5.788610e+12     5.788610e+12          1.51   

   Minimum Time  Average Iteration  Minimum Iteration  
0          1.35              34.15               28.0  
1          0.63              25.15               10.0  
2          0.68              23.90               11.0  
3          0.92              30.20               18.0  
boston.csv
  Initialization  Average Inertia  Minimum Inertia  Average Time  \
0         random       2812654.07       1475549.48          0.08   
1      k-means++       1572564.77       1475549.48          0.07   
2           orss       1547677.65       1475549.48          0.07   
3            coc       1812654.49       1475612.32          0.07   

   Minimum Tim

In [12]:
for i, df in enumerate(dfs):
    name, ext = splitext(files[i])
    file = join(outdir, name+'_out.csv')
    df.to_csv(file, index=False)